In [1]:
import numpy as np
import pandas as pd

In [2]:
#item_category_df = pd.read_csv("./data/item_categories.csv")
items_df = pd.read_csv("./data/items.csv")
#shops_df = pd.read_csv("./data/shops.csv")

train_df = pd.read_csv("./data/sales_train.csv")
#test_df = pd.read_csv("./data/test.csv")

#sample_df = pd.read_csv("./data/sample_submission.csv")

In [3]:
# date_df, 컬럼은 일, 월, 년도
def getDayDF():
    return pd.DataFrame(train_df["date"].str.split(".").tolist(),  columns = ['day','month','year'])

In [4]:
def get2DateDF():
    date_df = getDayDF()
    DF = pd.merge(train_df, date_df, left_index=True, right_index=True)
    del date_df
    # 0~33까지 년/월에 매핑
    DF['newDate'] = DF['year'].map(str) + DF['month']
    DF['dateCount'] = DF['newDate']
    DF['dateCount'].replace(DF['newDate'].unique().tolist(), np.arange(0, DF['newDate'].nunique()).tolist(), inplace=True)
    DF = DF.drop(['year', 'day', 'month'], axis=1)
    return DF

In [5]:
# 반환값 : 
def getDateDF():
    # split 되어있는 DataFrame Merge
    date_df = getDayDF()
    DF = pd.merge(train_df, date_df, left_index=True, right_index=True)
    del date_df
    # 0~33까지 년/월에 매핑
    DF['newDate'] = DF['year'].map(str) + DF['month']
    DF['dateCount'] = DF['newDate']
    DF['dateCount'].replace(DF['newDate'].unique().tolist(), np.arange(0, DF['newDate'].nunique()).tolist(), inplace=True)
    # Memory 부족 현상때문에 임시로 만드는 영역
    DF =  DF.drop(['day', 'month', 'newDate', 'year', 'date'], axis=1)
    return DF

In [10]:
# 반환값 : Item마다 0~33개월까지 판매량 메트릭스
def getMatrixDateDF():
    date_df = getDayDF()
    DF = pd.merge(train_df, date_df, left_index=True, right_index=True)
    del date_df
    # 0~33까지 년/월에 매핑
    DF['newDate'] = DF['year'].map(str) + DF['month']
    DF['dateCount'] = DF['newDate']
    DF['dateCount'].replace(DF['newDate'].unique().tolist(), np.arange(0, DF['newDate'].nunique()).tolist(), inplace=True)
    DF = DF.drop(['date', 'date_block_num', 'day', 'month', 'year', 'newDate', 'shop_id', 'item_price'], axis=1)
    DF = DF.groupby(['dateCount', 'item_id']).sum().unstack()
    DF[DF.isnull()] = 0
    return df_temp1.reset_index()

In [4]:
# 반환값 : item category와 날짜, train_df 셋을 merge
def getCategoryDF():
    date_df = getDayDF()
    DF = pd.merge(train_df, date_df, left_index=True, right_index=True)
    del date_df
    # 0~33까지 년/월에 매핑
    DF['newDate'] = DF['year'].map(str) + DF['month']
    DF['dateCount'] = DF['newDate']
    DF['dateCount'].replace(DF['newDate'].unique().tolist(), np.arange(0, DF['newDate'].nunique()).tolist(), inplace=True)
    df_temp1 = pd.merge(DF, items_df.drop('item_name', axis=1), on='item_id', left_index=True, right_index=False)
    return df_temp1

In [ ]:
# 반환값 : item category마다 0~33개월까지 판매량 메트릭스
def getMatrixCategoryDF():
    date_df = getDayDF()
    DF = pd.merge(train_df, date_df, left_index=True, right_index=True)
    del date_df
    # 0~33까지 년/월에 매핑
    DF['newDate'] = DF['year'].map(str) + DF['month']
    DF['dateCount'] = DF['newDate']
    DF['dateCount'].replace(DF['newDate'].unique().tolist(), np.arange(0, DF['newDate'].nunique()).tolist(), inplace=True)
    df_temp1 = pd.merge(DF, items_df.drop('item_name', axis=1), on='item_id', left_index=True, right_index=False)
    df_temp1 = df_temp1.drop(['date_block_num', 'date', 'shop_id', 'item_price', 'day', 'month', 'year', 'newDate', 'item_id'], axis=1)
    df_temp1 = df_temp1.groupby(['item_category_id', 'dateCount']).sum().unstack()
    df_temp1[df_temp1.isnull()] = 0
    return df_temp1.reset_index()

### MAX Value

In [6]:
# 반환 값 : 아이템 별로 최대로 팔리는 달 수와 팔린 값
def getMaxValueMatrix():
    # 날짜와 train_df 합쳐진 dataFrame을 불러옴
    maDF = getDateDF()
    # item 별로 달마다 판매한 개수 dataFrame
    #maDF = maDF.drop(['date_block_num', 'shop_id', 'date', 'item_price', 'day', 'month', 'year', 'newDate'], axis=1)
    maDF = maDF.drop(['date_block_num', 'shop_id', 'item_price'], axis=1)
    maDF = maDF.groupby(['item_id', 'dateCount']).sum().unstack().T
    maDF[maDF.isnull()] = 0
    maDF = maDF.reset_index().drop('level_0', axis=1)
    maDF = maDF.drop('dateCount', axis=1)
    maDF = maDF.T
    
    # 아이템별로 최댓값인 index와 value를 뽑아냄
    df_maxMonth= maDF.idxmax(axis=1).to_frame().reset_index()
    df_maxValue = maDF.max(axis=1).to_frame().reset_index()
    
    del maDF
    
    df_maxMonth = df_maxMonth.T.reset_index().T[1:]
    df_maxValue = df_maxValue.T.reset_index().T[1:]
    df_maxMonth['item_id'] = df_maxMonth[0]
    df_maxMonth['maxMonth'] = df_maxMonth[1]
    df_maxMonth = df_maxMonth.drop([0, 1], axis=1)
    df_maxValue['item_id'] = df_maxValue[0]
    df_maxValue['maxValue'] = df_maxValue[1]
    df_maxValue = df_maxValue.drop([0, 1], axis=1)
    
    df_resultMax = pd.merge(df_maxValue, df_maxMonth, on='item_id', left_index=True, right_index=True)
    del df_maxMonth
    del df_maxValue
    return df_resultMax

In [7]:
temp1 = getDateDF()
temp2 = getMaxValueMatrix()
mergedDF = pd.merge(temp1, temp2, on='item_id', sort=True, left_index=True, right_index=False)
del temp1
del temp2
del mergedDF['date_block_num']
del mergedDF['item_price']

# Train data를 위한 데이터를 작성
mergedDF['degreeMonth'] = (np.absolute((mergedDF['maxMonth'] - mergedDF['dateCount']).replace({0 : 1})).astype(np.float16).apply(lambda x : np.log(x)))
mergedDF['degreeMonth'] = mergedDF['maxValue'] / mergedDF['degreeMonth'].replace({0:1})
#Test data를 위한 항목을 작성
mergedDF['ForTestDegreeMonth'] = (np.absolute((mergedDF['maxMonth'] - 34).replace({0 : 1})).astype(np.float16).apply(lambda x : np.log(x)))
mergedDF['ForTestDegreeMonth'] = mergedDF['maxValue'] / mergedDF['ForTestDegreeMonth'].replace({0:1})
mergedDF = mergedDF.drop(['item_cnt_day', 'maxMonth'], axis=1)

In [8]:
mergedDF.head()

,shop_id,item_id,dateCount,maxValue,degreeMonth,ForTestDegreeMonth
0,54,0,20,1,1,0.378923
1,55,1,15,2,2,0.679247
1,55,1,15,2,2,0.679247
1,55,1,18,2,1.82048,0.679247
1,55,1,19,2,1.4427,0.679247


In [9]:
df_date = get2DateDF()
df_date = df_date.drop(['date_block_num', 'item_price', 'item_cnt_day', 'newDate'], axis=1)
resultDF = pd.merge(df_date, mergedDF, on=['shop_id', 'item_id', 'dateCount'], left_index=True, right_index=False)

MemoryError: 